## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import r2_score

from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.options.display.float_format = '{:.2f}'.format

## Importing Dataset

In [3]:
url = 'https://raw.githubusercontent.com/datasets/covid-19/main/data/time-series-19-covid-combined.csv'
df = pd.read_csv(url)

In [4]:
df

,Date,Country/Region,Province/State,Confirmed,Recovered,Deaths
0,2020-01-22,Afghanistan,NaN,0,0.00,0
1,2020-01-23,Afghanistan,NaN,0,0.00,0
2,2020-01-24,Afghanistan,NaN,0,0.00,0
3,2020-01-25,Afghanistan,NaN,0,0.00,0
4,2020-01-26,Afghanistan,NaN,0,0.00,0
...,...,...,...,...,...,...
104008,2021-02-01,Zimbabwe,NaN,33548,26583.00,1234
104009,2021-02-02,Zimbabwe,NaN,33814,26794.00,1254
104010,2021-02-03,Zimbabwe,NaN,33964,27391.00,1269
104011,2021-02-04,Zimbabwe,NaN,34171,27759.00,1288


In [5]:
df.columns

Index(['Date', 'Country/Region', 'Province/State', 'Confirmed', 'Recovered',
       'Deaths'],
      dtype='object')

## Generating Country Lists

In [6]:
countries = list(set(df['Country/Region']))

In [7]:
print(countries)

['Tanzania', 'Netherlands', 'Maldives', 'China', 'Greece', 'Costa Rica', 'Timor-Leste', 'Liberia', 'Armenia', 'Croatia', 'Poland', 'Lesotho', 'Albania', 'Colombia', 'Kazakhstan', 'Yemen', 'Lithuania', 'Peru', 'Nigeria', 'Pakistan', 'Tunisia', 'Uruguay', 'Gambia', 'Togo', 'Bhutan', 'Malawi', 'Chad', 'Sierra Leone', 'Holy See', 'Uganda', 'Syria', 'Israel', 'Zimbabwe', 'Mongolia', 'Cambodia', 'Portugal', 'Samoa', 'Slovenia', 'Papua New Guinea', 'Guatemala', 'Denmark', 'United Kingdom', 'Belarus', 'Seychelles', 'Congo (Brazzaville)', 'Georgia', 'Luxembourg', 'Ghana', 'Slovakia', 'Taiwan*', 'Guinea', 'Somalia', 'Burundi', 'Cameroon', 'Comoros', 'Saudi Arabia', 'Latvia', 'Ireland', 'Trinidad and Tobago', 'Panama', 'Indonesia', 'Spain', 'Iceland', 'Finland', 'Haiti', 'Japan', 'Jordan', 'Barbados', 'Liechtenstein', 'Romania', 'Korea, South', 'Fiji', 'France', 'Venezuela', 'Algeria', 'Qatar', 'Bahamas', 'Monaco', 'Namibia', 'Ecuador', 'Czechia', 'Turkey', 'Bahrain', 'Botswana', 'Germany', 'Domi

In [8]:
countries.sort()

In [9]:
countries

['Afghanistan',
 'Albania',
 'Algeria',
 'Andorra',
 'Angola',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burma',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Congo (Brazzaville)',
 'Congo (Kinshasa)',
 'Costa Rica',
 "Cote d'Ivoire",
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czechia',
 'Denmark',
 'Diamond Princess',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Fiji',
 'Finland',
 'France',
 'Gabon',
 'Gambia',
 'Georgia',
 'Germany',
 'Ghana',
 'Greece',
 'Grenada',
 'Guatemala',
 'Guinea',
 'Guinea-Bissau',
 'Guyana',

In [10]:
print("Total Number of Countries : ", len(countries))

Total Number of Countries :  192


## Confirmed Case Prediction

In [11]:
PredictedConfirmed = pd.DataFrame(columns=['Country', 'Confirmed', 'PredictedConfirmed'])
from fbprophet import Prophet

days =  60

In [12]:
def rename_func(dataframe):
    columns = dataframe.columns
    dataframe = dataframe.rename(columns= {columns[0]: 'ds', columns[1]: 'y'})
    return dataframe

In [13]:
for index,country in enumerate(countries):
    
    selected_country = df[df['Country/Region'] == country]
    selected_country['CurrentCase'] = selected_country['Confirmed'] - selected_country['Recovered'] - selected_country['Deaths']
    
    confirmed = selected_country.groupby('Date').sum()['Confirmed'].reset_index()
    recovered = selected_country.groupby('Date').sum()['Recovered'].reset_index()
    deaths = selected_country.groupby('Date').sum()['Deaths'].reset_index()
    current_case = selected_country.groupby('Date').sum()['CurrentCase'].reset_index()
    
    confirmed = rename_func(confirmed)
    recovered = rename_func(recovered)
    deaths = rename_func(deaths)
    current_case = rename_func(current_case)
    

    model = Prophet(daily_seasonality=False, yearly_seasonality=False)
    model.add_seasonality(name="Monthly", period=30.42, fourier_order=25)
    model.fit(confirmed.iloc[int(confirmed.shape[0]/2):])
    
    future_dates = model.make_future_dataframe(days)
    prediction = model.predict(future_dates)
    
    approximation = prediction[['ds', 'yhat_lower', 'yhat', 'yhat_upper']]
    
    confirmed_rows = confirmed.shape[0]
    prediction_rows = approximation.shape[0]
    
    
    current_value = confirmed.iloc[confirmed_rows-1]['y']
    predicted_value = float( '{:.2f}'.format( approximation.iloc[prediction_rows-1]['yhat'] ) )
    
    if current_value < 10000:
        output = output = '{index}. Country : {a}, Confirmed = {b}, Predicted (After {x} days) = {c}'.format(index=index+1, a=country, b=current_value, x=days, c = predicted_value)    
        print('-----------------------------------------------------------------------------------------------------------------------')
        print(output)
        print("SKIPPED: Currently Confirmed Cases is less than 10000. ")
        print('-----------------------------------------------------------------------------------------------------------------------\n')
       
    else:    
        output = '{index}. Country : {a}, Confirmed = {b}, Predicted (After {x} days) = {c}'.format(index=index+1, a=country, b=current_value, x=days, c = predicted_value)    
        print(output)
        print("Data Inserted In Dataset Successfully. ")
        PredictedConfirmed = PredictedConfirmed.append({'Country': country, 'Confirmed': current_value, 'PredictedConfirmed': predicted_value}, ignore_index=True)    
        print()

1. Country : Afghanistan, Confirmed = 55265, Predicted (After 60 days) = 62247.43
Data Inserted In Dataset Successfully. 

2. Country : Albania, Confirmed = 83082, Predicted (After 60 days) = 119081.56
Data Inserted In Dataset Successfully. 

3. Country : Algeria, Confirmed = 108629, Predicted (After 60 days) = 123729.36
Data Inserted In Dataset Successfully. 

4. Country : Andorra, Confirmed = 10172, Predicted (After 60 days) = 13916.66
Data Inserted In Dataset Successfully. 

5. Country : Angola, Confirmed = 20030, Predicted (After 60 days) = 24622.04
Data Inserted In Dataset Successfully. 

-----------------------------------------------------------------------------------------------------------------------
6. Country : Antigua and Barbuda, Confirmed = 277, Predicted (After 60 days) = 368.3
SKIPPED: Currently Confirmed Cases is less than 10000. 
-----------------------------------------------------------------------------------------------------------------------

7. Country : Arge

43. Country : Cote d'Ivoire, Confirmed = 29295, Predicted (After 60 days) = 40270.85
Data Inserted In Dataset Successfully. 

44. Country : Croatia, Confirmed = 234702, Predicted (After 60 days) = 279286.69
Data Inserted In Dataset Successfully. 

45. Country : Cuba, Confirmed = 31190, Predicted (After 60 days) = 54995.88
Data Inserted In Dataset Successfully. 

46. Country : Cyprus, Confirmed = 31512, Predicted (After 60 days) = 51808.2
Data Inserted In Dataset Successfully. 

47. Country : Czechia, Confirmed = 1021477, Predicted (After 60 days) = 1561621.26
Data Inserted In Dataset Successfully. 

48. Country : Denmark, Confirmed = 201458, Predicted (After 60 days) = 276999.51
Data Inserted In Dataset Successfully. 

-----------------------------------------------------------------------------------------------------------------------
49. Country : Diamond Princess, Confirmed = 712, Predicted (After 60 days) = 712.0
SKIPPED: Currently Confirmed Cases is less than 10000. 
------------

86. Country : Italy, Confirmed = 2611659, Predicted (After 60 days) = 3477299.99
Data Inserted In Dataset Successfully. 

87. Country : Jamaica, Confirmed = 16513, Predicted (After 60 days) = 21515.47
Data Inserted In Dataset Successfully. 

88. Country : Japan, Confirmed = 401858, Predicted (After 60 days) = 714060.61
Data Inserted In Dataset Successfully. 

89. Country : Jordan, Confirmed = 332990, Predicted (After 60 days) = 396213.39
Data Inserted In Dataset Successfully. 

90. Country : Kazakhstan, Confirmed = 240983, Predicted (After 60 days) = 307131.45
Data Inserted In Dataset Successfully. 

91. Country : Kenya, Confirmed = 101534, Predicted (After 60 days) = 109783.26
Data Inserted In Dataset Successfully. 

92. Country : Korea, South, Confirmed = 80524, Predicted (After 60 days) = 114977.88
Data Inserted In Dataset Successfully. 

93. Country : Kosovo, Confirmed = 61688, Predicted (After 60 days) = 78025.14
Data Inserted In Dataset Successfully. 

94. Country : Kuwait, Confi

-----------------------------------------------------------------------------------------------------------------------
128. Country : Niger, Confirmed = 4589, Predicted (After 60 days) = 7179.43
SKIPPED: Currently Confirmed Cases is less than 10000. 
-----------------------------------------------------------------------------------------------------------------------

129. Country : Nigeria, Confirmed = 137654, Predicted (After 60 days) = 221332.94
Data Inserted In Dataset Successfully. 

130. Country : North Macedonia, Confirmed = 94196, Predicted (After 60 days) = 112617.88
Data Inserted In Dataset Successfully. 

131. Country : Norway, Confirmed = 64483, Predicted (After 60 days) = 92905.46
Data Inserted In Dataset Successfully. 

132. Country : Oman, Confirmed = 135041, Predicted (After 60 days) = 145411.52
Data Inserted In Dataset Successfully. 

133. Country : Pakistan, Confirmed = 553128, Predicted (After 60 days) = 679333.22
Data Inserted In Dataset Successfully. 

134. Count

167. Country : Sweden, Confirmed = 588062, Predicted (After 60 days) = 882889.79
Data Inserted In Dataset Successfully. 

168. Country : Switzerland, Confirmed = 531873, Predicted (After 60 days) = 684752.88
Data Inserted In Dataset Successfully. 

169. Country : Syria, Confirmed = 14338, Predicted (After 60 days) = 19529.9
Data Inserted In Dataset Successfully. 

-----------------------------------------------------------------------------------------------------------------------
170. Country : Taiwan*, Confirmed = 923, Predicted (After 60 days) = 1130.89
SKIPPED: Currently Confirmed Cases is less than 10000. 
-----------------------------------------------------------------------------------------------------------------------

171. Country : Tajikistan, Confirmed = 13308, Predicted (After 60 days) = 13373.9
Data Inserted In Dataset Successfully. 

-----------------------------------------------------------------------------------------------------------------------
172. Country : T

In [14]:
PredictedConfirmed

,Country,Confirmed,PredictedConfirmed
0,Afghanistan,55265,62247.43
1,Albania,83082,119081.56
2,Algeria,108629,123729.36
3,Andorra,10172,13916.66
4,Angola,20030,24622.04
...,...,...,...
128,Uzbekistan,78989,82147.24
129,Venezuela,129231,154901.17
130,West Bank and Gaza,161559,203595.95
131,Zambia,60427,119932.72


In [15]:
PredictedConfirmed['Ratio'] = PredictedConfirmed['PredictedConfirmed'] / PredictedConfirmed['Confirmed']

In [16]:
PredictedConfirmed

,Country,Confirmed,PredictedConfirmed,Ratio
0,Afghanistan,55265,62247.43,1.13
1,Albania,83082,119081.56,1.43
2,Algeria,108629,123729.36,1.14
3,Andorra,10172,13916.66,1.37
4,Angola,20030,24622.04,1.23
...,...,...,...,...
128,Uzbekistan,78989,82147.24,1.04
129,Venezuela,129231,154901.17,1.20
130,West Bank and Gaza,161559,203595.95,1.26
131,Zambia,60427,119932.72,1.98


In [17]:
# if days == 30:
#     PredictedConfirmed.to_csv('/home/zayed/Desktop/Output/predicted_confirmed_30_days.csv', index=False)
# if days == 60:
#     PredictedConfirmed.to_csv('/home/zayed/Desktop/Output/predicted_confirmed_60_days.csv', index=False)
# if days == 90:
#     PredictedConfirmed.to_csv('/home/zayed/Desktop/Output/predicted_confirmed_90_days.csv', index=False)

## Death Prediction

#### Creating Empty Dataframe

In [18]:
PredictedDeaths = pd.DataFrame(columns=['Country', 'Deaths', 'PredictedDeaths'])
from fbprophet import Prophet

#### Forecasting & Storing Information of all countries

In [ ]:
for index,country in enumerate(countries):
    
    selected_country = df[df['Country/Region'] == country]
    selected_country['CurrentCase'] = selected_country['Confirmed'] - selected_country['Recovered'] - selected_country['Deaths']
    
    confirmed = selected_country.groupby('Date').sum()['Confirmed'].reset_index()
    recovered = selected_country.groupby('Date').sum()['Recovered'].reset_index()
    deaths = selected_country.groupby('Date').sum()['Deaths'].reset_index()
    current_case = selected_country.groupby('Date').sum()['CurrentCase'].reset_index()
    
    confirmed = rename_func(confirmed)
    recovered = rename_func(recovered)
    deaths = rename_func(deaths)
    current_case = rename_func(current_case)
    

    model = Prophet(daily_seasonality=False, yearly_seasonality=False, holidays=None,  interval_width=0.95, growth='linear')
    model.add_seasonality(name="Monthly", period=30.42, fourier_order=25)
    model.fit(deaths.iloc[int(deaths.shape[0]/2):])
    
    future_dates = model.make_future_dataframe(days)
    prediction = model.predict(future_dates)
    
    approximation = prediction[['ds', 'yhat_lower', 'yhat', 'yhat_upper']]
    
    death_rows = deaths.shape[0]
    prediction_rows = approximation.shape[0]
    
    
    current_value = deaths.iloc[death_rows-1]['y']
    predicted_value = float( '{:.2f}'.format( approximation.iloc[prediction_rows-1]['yhat'] ) )
    
    if current_value < 1000:
        output = output = '{index}. Country : {a}, Deaths = {b}, Deaths Prediction(After {x} days) = {c}'.format(index=index+1, a=country, b=current_value, x=days, c = predicted_value)    
        print('-----------------------------------------------------------------------------------------------------------------------')
        print(output)
        print("SKIPPED: Current Deaths is less than 1000.  ")
        print('-----------------------------------------------------------------------------------------------------------------------\n')
    else:    
        output = '{index}. Country : {a}, Deaths = {b}, Deaths Prediction (After {x} days) = {c}'.format(index=index+1, a=country, b=current_value, x=days, c = predicted_value)    
        print(output)
        print("Data Inserted In Dataset Successfully. ")
        PredictedDeaths = PredictedDeaths.append({'Country': country, 'Deaths': current_value, 'PredictedDeaths': predicted_value}, ignore_index=True)    
        print()

1. Country : Afghanistan, Deaths = 2407, Deaths Prediction (After 60 days) = 2841.75
Data Inserted In Dataset Successfully. 

2. Country : Albania, Deaths = 1433, Deaths Prediction (After 60 days) = 1798.46
Data Inserted In Dataset Successfully. 

3. Country : Algeria, Deaths = 2909, Deaths Prediction (After 60 days) = 3172.05
Data Inserted In Dataset Successfully. 

-----------------------------------------------------------------------------------------------------------------------
4. Country : Andorra, Deaths = 104, Deaths Prediction(After 60 days) = 133.85
SKIPPED: Current Deaths is less than 1000.  
-----------------------------------------------------------------------------------------------------------------------

-----------------------------------------------------------------------------------------------------------------------
5. Country : Angola, Deaths = 474, Deaths Prediction(After 60 days) = 594.12
SKIPPED: Current Deaths is less than 1000.  
------------------------

-----------------------------------------------------------------------------------------------------------------------
34. Country : Central African Republic, Deaths = 63, Deaths Prediction(After 60 days) = 63.0
SKIPPED: Current Deaths is less than 1000.  
-----------------------------------------------------------------------------------------------------------------------

-----------------------------------------------------------------------------------------------------------------------
35. Country : Chad, Deaths = 124, Deaths Prediction(After 60 days) = 151.4
SKIPPED: Current Deaths is less than 1000.  
-----------------------------------------------------------------------------------------------------------------------

36. Country : Chile, Deaths = 18808, Deaths Prediction (After 60 days) = 22235.0
Data Inserted In Dataset Successfully. 

37. Country : China, Deaths = 4822, Deaths Prediction (After 60 days) = 4892.94
Data Inserted In Dataset Successfully. 

38. Country : Col

-----------------------------------------------------------------------------------------------------------------------
65. Country : Gambia, Deaths = 132, Deaths Prediction(After 60 days) = 137.31
SKIPPED: Current Deaths is less than 1000.  
-----------------------------------------------------------------------------------------------------------------------

66. Country : Georgia, Deaths = 3258, Deaths Prediction (After 60 days) = 4585.07
Data Inserted In Dataset Successfully. 

67. Country : Germany, Deaths = 61324, Deaths Prediction (After 60 days) = 108305.65
Data Inserted In Dataset Successfully. 

-----------------------------------------------------------------------------------------------------------------------
68. Country : Ghana, Deaths = 449, Deaths Prediction(After 60 days) = 529.3
SKIPPED: Current Deaths is less than 1000.  
-----------------------------------------------------------------------------------------------------------------------

69. Country : Greece, Dea

103. Country : Lithuania, Deaths = 2908, Deaths Prediction (After 60 days) = 5029.36
Data Inserted In Dataset Successfully. 

-----------------------------------------------------------------------------------------------------------------------
104. Country : Luxembourg, Deaths = 590, Deaths Prediction(After 60 days) = 772.03
SKIPPED: Current Deaths is less than 1000.  
-----------------------------------------------------------------------------------------------------------------------

-----------------------------------------------------------------------------------------------------------------------
105. Country : MS Zaandam, Deaths = 2, Deaths Prediction(After 60 days) = 2.0
SKIPPED: Current Deaths is less than 1000.  
-----------------------------------------------------------------------------------------------------------------------

-----------------------------------------------------------------------------------------------------------------------
106. Country : Madaga

130. Country : North Macedonia, Deaths = 2908, Deaths Prediction (After 60 days) = 3612.18
Data Inserted In Dataset Successfully. 

-----------------------------------------------------------------------------------------------------------------------
131. Country : Norway, Deaths = 582, Deaths Prediction(After 60 days) = 838.13
SKIPPED: Current Deaths is less than 1000.  
-----------------------------------------------------------------------------------------------------------------------

132. Country : Oman, Deaths = 1532, Deaths Prediction (After 60 days) = 1594.06
Data Inserted In Dataset Successfully. 

133. Country : Pakistan, Deaths = 11914, Deaths Prediction (After 60 days) = 14857.27
Data Inserted In Dataset Successfully. 

134. Country : Panama, Deaths = 5426, Deaths Prediction (After 60 days) = 7928.15
Data Inserted In Dataset Successfully. 

-----------------------------------------------------------------------------------------------------------------------
135. Country

In [ ]:
PredictedDeaths

In [ ]:
PredictedDeaths['Ratio'] = PredictedDeaths['PredictedDeaths'] / PredictedDeaths['Deaths']

In [ ]:
PredictedDeaths

### Exporting The Dataframe

In [ ]:
# if days == 30:
#     PredictedDeaths.to_csv('/home/zayed/Desktop/Output/predicted_deaths_30_days.csv', index=False)
# if days == 60:
#     PredictedDeaths.to_csv('/home/zayed/Desktop/Output/predicted_deaths_60_days.csv', index=False)
# if days == 90:
#     PredictedDeaths.to_csv('/home/zayed/Desktop/Output/predicted_deaths_90_days.csv', index=False)